# calibrate_simulation

> A Metamodel-Based General-purpose Calibration Tool for Simulation Models

In [7]:
#| default_exp calibrate_simulation

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
#| hide
from nbdev.showdoc import *

In [9]:
#| export
from enum import Enum
class OptimizerType(Enum):
    GUROBI = 1
    OR_TOOLS = 2

In [10]:
#| export
import random
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import models 
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint

import gurobipy as gp
from gurobipy import GRB

from ortools.linear_solver import pywraplp




class CalibrateSimulation:


    def __init__(self,optimizer_type : OptimizerType):
        self.num_hidden_nodes = 1000
        self.optimizer_type = optimizer_type
        return

    def train_model(self, x_training, y_training, X_validation, y_validation):
        self.num_input_cols = x_training.shape[1]
        self.num_ouptup_cols = y_training.shape[1]

        NN_model = models.Sequential()
        NN_model.add(layers.Dense(self.num_hidden_nodes,activation='relu'))  #One hidden layer with 1000 nodes
        NN_model.add(layers.Dense(self.num_ouptup_cols))
        NN_model.compile(optimizer='adam', loss='mean_absolute_error')
        chk = ModelCheckpoint('NN_model',monitor='val_loss',save_best_only=True,mode='min',verbose=1)
        self.history = NN_model.fit(x_training,y_training,epochs=100,callbacks=[chk],validation_data=(X_validation,y_validation))

        self.NN_model = tf.keras.models.load_model('NN_model')

        self.weights_1 = np.array(self.NN_model.get_weights()[0])
        self.weights_2 = np.array(self.NN_model.get_weights()[1])
        self.weights_3 = np.array(self.NN_model.get_weights()[2])
        self.weights_4 = np.array(self.NN_model.get_weights()[3])

    def solve_optimization(self, target_val, lower_bound, upper_bound):
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        if self.optimizer_type == OptimizerType.GUROBI :
            return self._solve_optimization_gurobi(target_val)
        else :
            return self._solve_optimization_or_tools(target_val)

        
    def _solve_optimization_gurobi(self, target_val):
        # Create initial model
        model = gp.Model('project')
        model.Params.LogToConsole = 0
        model.Params.MIPFocus = 1
        model.setParam('MIPGap', 0.000001)

        #Index

        I = self.num_input_cols
        J = self.num_ouptup_cols
        L = self.num_hidden_nodes

        #Parameters

        M = self.num_hidden_nodes
        h = target_val
        w1 = self.weights_1
        b1 = self.weights_2
        w2 = self.weights_3
        b2 = self.weights_4

        #Decision Variables

        x = model.addVars(I,vtype=GRB.CONTINUOUS,name='x',lb=self.lower_bound,ub=self.upper_bound)  #the value of parameter i
        y = model.addVars(L, vtype=GRB.CONTINUOUS,name='y')  #the hidden node value in output j for node l
        z = model.addVars(J, vtype=GRB.CONTINUOUS,name='z')  #the model value for output j
        u = model.addVars(L, vtype=GRB.BINARY,name='u')  #the binary indicator for activation of l th hidden node
        d = model.addVars(J, vtype=GRB.CONTINUOUS,name='d')  #the difference value for output j

        # Constraints:

        model.addConstrs((gp.quicksum(w1[i, l] * x[i] for i in range(I)) + b1[l] <= y[l] for l in range(L)),name='FirstConsts')
        model.addConstrs((gp.quicksum(w1[i, l] * x[i] for i in range(I)) + b1[l] + M * (1 - u[l]) >= y[l] for l in range(L)),name='SecondConsts')
        model.addConstrs((y[l] <= M * u[l] for l in range(L)),name='ThirdCOnsts')
        model.addConstrs((gp.quicksum(y[l] * w2[l, j] for l in range(L)) + b2[j] == z[j]for j in range(J)),name='FourthConsts')
        model.addConstrs((d[j] >= z[j] - h[j] for j in range(J)),name='FifthConsts')
        model.addConstrs((d[j] >= h[j] - z[j] for j in range(J)),name='SixthConsts')

        # Set global sense for ALL objectives
        model.ModelSense = GRB.MINIMIZE

        obj = gp.quicksum(d[j] for j in range(J))
        model.setObjective(obj)

        # Optimize

        model.optimize()

        parameter_list = []
        for i in range(I):
            parameter_list.append(x[i].x)

        output_list = []
        for i in range(J):
            output_list.append(z[i].x)
        """
        for v in model.getVars():
            print('%s %g' % (v.varName, v.x))
                
        print('Obj: %g' % obj.getValue())
        """

        return parameter_list, output_list
    
    def _solve_optimization_or_tools(self, target_val):
        solver = pywraplp.Solver.CreateSolver('SCIP')

        mip_gap = 0.000001

        #model = pywraplp.Solver('model', pywraplp.Solver.SCIP_MIXED_INTEGER_PROGRAMMING)

        solverParams = pywraplp.MPSolverParameters()
        solverParams.SetDoubleParam(solverParams.RELATIVE_MIP_GAP, mip_gap)

        #Index

        I = self.num_input_cols
        J = self.num_ouptup_cols
        L = self.num_hidden_nodes

        #Parameters

        M = self.num_hidden_nodes
        h = target_val
        w1 = self.weights_1
        b1 = self.weights_2
        w2 = self.weights_3
        b2 = self.weights_4

        lb = [1.0, 0.5, 5.0, 1.0, 0.4, 390.0]
        ub = [2.5, 1.0, 10.0, 5.0, 0.8, 760.0]
        infinity = solver.infinity()
        x_vars = {}
        y_vars = {}
        z_vars = {}
        u_vars = {}
        d_vars = {}

        for i in range(I):
            x_vars[i] = solver.NumVar(lb=self.lower_bound[i], ub=self.upper_bound[i], name=f'x[{i}]')
        for i in range(L):
            y_vars[i] = solver.NumVar(lb=0.0, ub=infinity, name=f'y[{i}]')
        for i in range(J):
            z_vars[i] = solver.NumVar(lb=0.0, ub=infinity, name=f'z[{i}]')
        for i in range(L):
            u_vars[i] = solver.BoolVar(name=f'u[{i}]')
        for i in range(J):
            d_vars[i] = solver.NumVar(lb=0.0, ub=infinity, name=f'd[{i}]')

        for l in range(L):
            constraint_expr = [w1[i,l] * x_vars[i] for i in range(I)]
            solver.Add(sum(constraint_expr) + b1[l] <= y_vars[l])

        for l in range(L):
            constraint_expr = [w1[i,l] * x_vars[i] for i in range(I)]
            solver.Add(sum(constraint_expr) + b1[l] + M * (1 - u_vars[l]) >= y_vars[l])

        for l in range(L):
            solver.Add(M * u_vars[l] >= y_vars[l])

        for j in range(J):
            constraint_expr = [w2[l,j] * y_vars[l] for l in range(L)]
            solver.Add(sum(constraint_expr) + b2[j] == z_vars[j])

        for j in range(J):
            solver.Add(z_vars[j] - h[j] <= d_vars[j])

        for j in range(J):
            solver.Add(h[j] - z_vars[j] <= d_vars[j])


        obj_expr = [d_vars[j] for j in range(J)]
        solver.Minimize(solver.Sum(obj_expr))

        status = solver.Solve()

        self.parameter_list = [x_vars[i].solution_value() for i in range(I)]


        self.output_list = [z_vars[j].solution_value() for j in range(J)]

        return self.parameter_list, self.output_list

In [11]:
#| hide
import nbdev; nbdev.nbdev_export()

## Testing

In [12]:
M = pd.read_csv('testing/data/output-seed_1337-30000.csv')
#NORMALIZATION
M.drop(M.columns[[8,12,13,16,17,18,21]], axis = 1, inplace= True)
for i in range(6,15):
    M.iloc[:,i] = (M.iloc[:,i] - M.iloc[:,i].min())/(M.iloc[:,i].max() - M.iloc[:,i].min())
    target_sample = M.sample(100)
M = M.drop(target_sample.index)
M.reset_index(inplace=True)
M.drop(M.columns[0], axis = 1, inplace=True)
M.to_csv('testing/data/OutputMinMaxScaledResults30000.csv')
target_sample.to_csv('testing/data/TargetSample.csv')

M_train = M.sample(frac=0.7,  random_state=1337)
M_rest = M.drop(M_train.index)
M_validation = M_rest.sample(frac=0.5,  random_state=1337)
M_test = M_rest.drop(M_validation.index)

X_training = np.array(M_train.iloc[:,:6])
X_validation = np.array(M_validation.iloc[:,:6])
X_test = np.array(M_test.iloc[:,:6])

Y_training = np.array(M_train.iloc[:,6:])
Y_validation = np.array(M_validation.iloc[:,6:])
Y_test = np.array(M_test.iloc[:,6:])

In [13]:
Target_Sample = pd.read_csv('testing/data/TargetSample.csv', index_col=0).iloc[:,[6,7,8,9,10,11,12,13,14]]
Target_Sample.reset_index(inplace=True)
Target_Sample.drop(Target_Sample.columns[0], axis = 1, inplace=True)
target_val = np.array(Target_Sample[0:1]).tolist()[0]

In [14]:
optimizer = CalibrateSimulation(OptimizerType.OR_TOOLS)
optimizer.train_model(X_training, Y_training, X_validation, Y_validation)


Epoch 1/100
648/655 [============================>.] - ETA: 0s - loss: 1.3731
Epoch 1: val_loss improved from inf to 0.86117, saving model to NN_model
INFO:tensorflow:Assets written to: NN_model\assets
655/655 [==============================] - 2s 2ms/step - loss: 1.3718 - val_loss: 0.8612
Epoch 2/100
652/655 [============================>.] - ETA: 0s - loss: 1.0440
Epoch 2: val_loss improved from 0.86117 to 0.81997, saving model to NN_model
INFO:tensorflow:Assets written to: NN_model\assets
655/655 [==============================] - 1s 2ms/step - loss: 1.0432 - val_loss: 0.8200
Epoch 3/100
647/655 [============================>.] - ETA: 0s - loss: 0.9683
Epoch 3: val_loss improved from 0.81997 to 0.51361, saving model to NN_model
INFO:tensorflow:Assets written to: NN_model\assets
655/655 [==============================] - 1s 2ms/step - loss: 0.9639 - val_loss: 0.5136
Epoch 4/100
645/655 [============================>.] - ETA: 0s - loss: 0.8392
Epoch 4: val_loss did not improve from 0.

In [15]:
sample_arrays = np.array(Target_Sample)
lower_bound = [1, 0.5, 5, 1, 0.4, 390]
upper_bound = [2.5, 1, 10, 5, 0.8,760]

results_param = {}
results_output = {}
for i in range(100):
    result = optimizer.solve_optimization(sample_arrays[i],lower_bound,upper_bound)
    results_param[i] = result[0]
    results_output[i] = result[1]
    print(f'{i} done')


0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done


In [16]:
results_param

{0: [1.9017864096281392, 0.5, 4.999999999999999, 1.0, 0.4, 593.1479655581693],
 1: [1.0459946605063444,
  1.0,
  10.0,
  4.29313000946394,
  0.799999999999999,
  760.0],
 2: [1.2126412929948818,
  1.0,
  10.0,
  3.0886420595672717,
  0.4188223665726387,
  619.0464880799664],
 3: [1.3633434320756341,
  0.5,
  5.000000000000019,
  1.0,
  0.4444223853961062,
  760.0],
 4: [1.5865158420909167,
  0.6666362128971505,
  10.0,
  3.6539687808126065,
  0.4,
  760.0],
 5: [1.1628844989832543,
  0.7491694691796429,
  8.650414648825722,
  3.90279154629549,
  0.8,
  760.0],
 6: [2.3612115214820024, 0.5, 4.999999999999999, 1.0, 0.4, 499.07415720124254],
 7: [1.0791888255906517, 1.0, 10.0, 4.220347182781768, 0.8, 760.0],
 8: [1.5368367393720535,
  1.0,
  5.000000000000001,
  0.9999999999999997,
  0.5305030943817475,
  404.60389274965627],
 9: [2.231117964517724,
  1.0,
  5.919179539874616,
  4.0276778839695275,
  0.39999999999999997,
  390.00000000000006],
 10: [1.9533838501536465, 0.5, 5.0, 3.6445486

In [17]:
results_output

{0: [0.7295953944730242,
  0.7202500368175964,
  0.6622022237694446,
  0.6816473657277933,
  0.6512359192886076,
  0.5741676409088042,
  0.6019786910197868,
  0.5456529480795654,
  0.5780326640736557],
 1: [0.0980250272076269,
  0.0762324454485575,
  0.038759590677005594,
  0.05767681129396809,
  0.046456458901943526,
  0.032617467939930186,
  0.0468036529680365,
  0.048651547522152484,
  0.05120587725984501],
 2: [0.2309789453651945,
  0.21327958858896964,
  0.15856903920067747,
  0.17635429465862362,
  0.1579816164758571,
  0.12676056338028172,
  0.14510949486157787,
  0.1331990880714687,
  0.14609011915918838],
 3: [0.37601020074621677,
  0.36130950582083815,
  0.28871068185593973,
  0.3103934205196494,
  0.28267869850330074,
  0.22154130526863633,
  0.2448730245626209,
  0.2219297091311608,
  0.23676736769662024],
 4: [0.5542105676985166,
  0.5415665853956831,
  0.4751090748589672,
  0.49639936750651853,
  0.4559041638903898,
  0.4062258766139171,
  0.4322678843226788,
  0.39161413